In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')

In [ ]:
data = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

In [ ]:
data.describe()

In [ ]:
plt.figure(figsize = (12,6))
sns.countplot(x = 'quality',data = data)

In [ ]:
data['wineClass'] = data['quality'].copy()
for index in range(len(data['wineClass'])):
    if data['wineClass'][index]>=6:data['wineClass'][index] = 'Good'
    else:data['wineClass'][index] = 'Bad'

In [ ]:
plt.figure(figsize = (12,6))
sns.countplot(x = 'wineClass',data = data)

In [ ]:
list(data.select_dtypes(include = ['object']).columns)

In [ ]:
Y = data['wineClass']
data.drop(['wineClass','quality'],axis = 1,inplace = True)

In [ ]:
 fig=plt.figure(figsize=(30, 60)) 
columns = 2 
rows = 6
column = list(data.columns)
col = 0
for i in range(1, columns*rows +1): 
    if col==len(column):break
    fig.add_subplot(rows, columns, i) 
    tab = data[column[col]]
    plt.hist(tab)
    plt.xlabel(column[col])
    col+=1
plt.show() 

In [ ]:
data.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [ ]:
logistic_reg = LogisticRegression()
random_forest = RandomForestClassifier()
decision_tree = DecisionTreeClassifier()
svm = SVC()

In [ ]:
Xtrain,xtest, Ytrain, ytest = train_test_split(data,Y,test_size = 0.2)

In [ ]:
def trainer(model):
    model.fit(Xtrain,Ytrain)
    return model.score(xtest,ytest)
models = [logistic_reg,random_forest,decision_tree,svm]
model_names =  ['Logistic Regression','Random Forest Classifier','Decision Tree Classifier','Support Vector Machines']

training_with_defaults = []
for model,model_name in zip(models,model_names):
    score = trainer(model)
    training_with_defaults.append('{} : {}'.format(model_name,score))
    print('{} : {}'.format(model_name,score))
    

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def gridsearch(model,params_grid):
    grid_search = GridSearchCV(estimator=model,param_grid = params_grid,cv = 3,verbose = 2,n_jobs = 4)
    grid_search.fit(Xtrain,Ytrain)
    return grid_search.best_params_
logistic_reg_params_grid = {'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                            'dual':[True,False],
                            'C':np.arange(11),
                            'fit_intercept':[True,False]
                            
                           }
random_forest_params_grid = {'n_estimators':[20,70,100],
                            'criterion':['gini','entropy'],
                             'max_depth':[None,5,10],
                             'min_samples_split':[2,7],
                             'min_samples_leaf':[1,2,4]
                            }
decision_tree_params_grid = {'criterion':['gini', 'entropy'],
                             'splitter' : ['best', 'random'],
                             'max_depth':[None,5,10,15],
                             'min_samples_split':[2,7],
                             'min_samples_leaf':[1,2,4]
                            }
svm_params_grid = {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']}
parameter_grids = [logistic_reg_params_grid,random_forest_params_grid,decision_tree_params_grid]
param_dict = {}
for model,model_name,paramgrid in zip(models,model_names,parameter_grids):
    if model_name=='Support Vector Machine':break
    parameters = gridsearch(model,paramgrid)
    param_dict[model_name] = parameters
print(param_dict)

In [ ]:
logistic_reg = LogisticRegression(C= 1,dual= False, fit_intercept= True, penalty= 'l2')
random_forest = RandomForestClassifier(criterion= 'entropy', max_depth= None, min_samples_leaf= 1, min_samples_split= 2, n_estimators= 100)
decision_tree = DecisionTreeClassifier(criterion= 'entropy', max_depth= 15, min_samples_leaf= 1, min_samples_split= 2, splitter = 'best')

In [ ]:
training_with_hypers = []
for model,model_name in zip(models,model_names):
    score = trainer(model)
    training_with_hypers.append('{} : {}'.format(model_name,score))
#     print('{} : {}'.format(model_name,score))

In [ ]:
print('Defaults : ')
for res in training_with_defaults:print(res)
print('\n\nTuned : ')
for res in training_with_hypers:print(res)